# CP 322 - Final Project

Aditya Chauhan (169027493)

### Dataset Choice

- [Student Performance Dataset](https://archive.ics.uci.edu/dataset/320/student+performance)

### Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

### Read Data

In [5]:
# Load the datasets
student_mat = pd.read_csv("student-mat.csv", sep=";")
student_por = pd.read_csv("student-por.csv", sep=";")

# Merge datasets on specified columns
merge_columns = [
    "school", "sex", "age", "address", "famsize", "Pstatus", 
    "Medu", "Fedu", "Mjob", "Fjob", "reason", "nursery", "internet"
]
merged_data = pd.merge(student_mat, student_por, on=merge_columns)

# Check the number of rows in the merged dataset
print(f"Number of students in merged dataset: {merged_data.shape[0]}")

# Save the merged dataset
merged_data.to_csv("student-merged.csv", index=False)

print("Merged dataset saved to 'student-merged.csv'.")

Number of students in merged dataset: 382
Merged dataset saved to 'student-merged.csv'.


## Data Quality Report